<a href="https://colab.research.google.com/github/Shrutika-Patil04/AI_Resume_Screening_Tool/blob/main/Resume_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

resume_folder = '/content/drive/My Drive/Colab Notebooks/Resumes'
resume_files = [f for f in os.listdir(resume_folder) if f.endswith(('.pdf', '.docx'))]

print("Found resumes:")
for file in resume_files:
    print(file)


Found resumes:
Pankaj BSA.docx
Bharatha BA Resume.docx
Bharat Arora_CV_PMP _ ERP1.docx
Brahma-Resume (BA).docx
BA - Navneet.docx
Amrinder Business Analyst.docx
Manohar Reddy.docx
Krishna Sr. BSA Resume.docx
Jagan S Iyer PM.docx
Gokul Selvam S PM.docx
Resume - Sr PM.docx
Kashyap K. Vora resume.docx
Randy Adams.docx
Krishna Kuruvella.docx
Rashmitha R.docx
Adelina_Erimia_PMP1.docx
Naren Sr.BA.docx
Amulya Komatineni.docx
Priya_Sharma.docx
Mounika_P.docx
Francis Gomes Resume.docx
Mohamad Jamil.docx
Mahesh_Kamath_PM (1).docx
Othman - Project Manager.docx
Ashok Jayakumar - PM.docx
Naren Sr.BA .docx
Naveen Arora.docx
Jimi_Desai_PM.docx
Kumar Raj.docx
Derik.docx
Manohar B.docx
B Shaker-Sr BSA-Scrum Master .docx
Atul_Mathur_Resume.docx
Ranjan_Project Manager-Scrum Master.docx
Ravi Reddy.docx
Reddemma_Lankipalle.docx
Manikanta P.docx
Gopi.docx
Naveen Kumar Bandela.docx
Raju Goduguchinta_Technical Program Manager.docx
Neha Mugghala.docx
BA Kiran.docx
Achyuth Resume_8.docx
Naresh V.docx
Ravi Pattar

In [ ]:
!pip install python-docx pdfplumber nltk scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.8 MB/s eta 0:00:00


In [ ]:
import docx
import pdfplumber

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_pdf(file_path):
    text = ''
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + '\n'
    return text

resume_texts = []

for file in resume_files:
    path = os.path.join(resume_folder, file)
    if file.endswith('.docx'):
        text = extract_text_from_docx(path)
    elif file.endswith('.pdf'):
        text = extract_text_from_pdf(path)
    resume_texts.append((file, text))


In [8]:
import nltk
import re
from nltk.corpus import stopwords

# Download only stopwords
nltk.download('stopwords', quiet=True)

# Define stopwords
stop_words = set(stopwords.words('english'))

# Clean text using simple tokenizer to avoid punkt_tab error
def clean_text(text):
    text = re.sub(r'\W+', ' ', text.lower())       # Remove non-alphanumeric
    words = text.split()                           # Simple split instead of word_tokenize
    words = [w for w in words if w not in stop_words and len(w) > 2]
    return ' '.join(words)

# Apply cleaning to all resumes
cleaned_resumes = [(filename, clean_text(text)) for filename, text in resume_texts]

# Print preview
for filename, cleaned in cleaned_resumes[:3]:  # Preview first 3
    print(f"Filename: {filename}\nCleaned Text: {cleaned[:300]}...\n")


Filename: Pankaj BSA.docx
Cleaned Text: pankaj business system analyst summary years experience business system analyst diverse domains like investment banking loans automobile commercial banking thorough knowledge fixed income including equity asset management bonds mutual funds securities depth knowledge software development life cycle ...

Filename: Bharatha BA Resume.docx
Cleaned Text: professional summary years experience business analyst working across various domains like finance extensive experience business analysis business processes business process flows gathering requirements development enhancement healthcare finance applications expertise conducting joint application de...

Filename: Bharat Arora_CV_PMP _ ERP1.docx
Cleaned Text: bharat arora pmp erp consultant contact 732 917 4097 summary passionate value driven consultant years experience leading cross functional teams plan build launch manage various world class applications worked projects business meetings client acr

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 👉 Enter your job description here
job_description = """
Looking for a data analyst with experience in Python, SQL, machine learning, and dashboarding using tools like Power BI or Tableau. Familiarity with statistics and communication skills is a must.
"""

# Combine JD and resumes
texts = [job_description] + [text for _, text in cleaned_resumes]

# Vectorize using TF-IDF
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(texts)

# Compute cosine similarity
job_vec = vectors[0]
resume_vecs = vectors[1:]
similarities = cosine_similarity(job_vec, resume_vecs).flatten()


In [11]:
import numpy as np

# Get top N matching resumes (e.g., top 5)
top_n = 5
top_indices = np.argsort(similarities)[::-1][:top_n]

print("📄 Top Matching Resumes with Accuracy Percentage:\n")
for idx in top_indices:
    filename = cleaned_resumes[idx][0]
    score = similarities[idx]
    accuracy_percent = round(score * 100, 2)  # Convert to percentage
    print(f"{filename} --> Match: {accuracy_percent}%")


📄 Top Matching Resumes with Accuracy Percentage:

RaviRaju_Resume.docx --> Match: 6.61%
Vijay Bhargav.docx --> Match: 5.74%
Yohan BSA.docx --> Match: 5.6%
Kashyap K. Vora resume.docx --> Match: 5.12%
Manohar Reddy.docx --> Match: 5.09%
